# Access Data

In [1]:
%let path=C:\Users\pestyl\OneDrive - SAS\github repos\SAS Case Studies and Workshops\SQL Case Study - TSA Claims;
libname sq "&path";

SAS Connection established. Subprocess id is 7420

7                                                          The SAS System                             15:27 Monday, January 25, 2021

34         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
34       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
35         
36         %let path=C:\Users\pestyl\OneDrive - SAS\github repos\SAS Case Studies and Workshops\SQL Case Study - TSA Claims;
37         libname sq "&path";
NOTE: Libref SQ was successfully assigned as follows: 
      Engine:        V9 
      Physical Name: C:\Users\pestyl\OneDrive - SAS\github repos\SAS Case Studies and Workshops\SQL Case Study - TSA Claims
38         
39         
40         ods html5 (id=saspy_internal) close;ods listing;
41         
8                                                          The SAS System                             15:27 Monday,

# Explore Data

#### 2.Preview the first 10 rows and descriptor portion of the following tables

In [2]:
proc sql outobs=10;
title "Table: CLAIMSRAW";
describe table sq.claimsraw;
select * 
    from sq.claimsraw;
title "Table: ENPLANEMENT2017";
describe table sq.enplanement2017;
select * 
    from sq.enplanement2017;
title "Table: BOARDING2013_2016";
describe table sq.boarding2013_2016;
select * 
    from sq.boarding2013_2016;
title;
quit;

Claim_Number,Date_Received,Incident_Date,Airport_Code,Airport_Name,Claim_Type,Claim_Site,Close_Amount,Disposition,StateName,State,County,City
2013042303294,23APR2013,04APR2013,,,Property Damage,Checked Baggage,.,,,,,
2013031302565,13MAR2013,13MAR2013,,,,,.,,,,,
2013031902651,11MAR2013,21FEB2013,,,Property Damage,Checked Baggage,.,,,,,
2013041503167,27MAR2013,08MAR2013,,,Property Damage,Checked Baggage,.,,,,,
2013040402988,22MAR2013,08FEB2013,,,,Checked Baggage,.,,,,,
2013040202938,19MAR2013,05MAR2013,,,Passenger Property Loss,Checked Baggage,.,,,,,
2013061904529,09MAY2013,11MAR2013,,,Passenger Property Loss,Checked Baggage,.,,,,,
2013043003398,30APR2013,30APR2013,,,Property Damage,Checkpoint,.,,,,,
2013061804491,07MAY2013,18MAR2013,,,Property Damage,Checked Baggage,.,,,,,
2013040803043,08APR2013,08APR2013,,,,,.,,,,,


**Summary**: 

a. Year is character in the enplanement table, numeric in the boarding table.                                  
b. The number of passengers is called enplanement in the enplanement table, and boarding in the boarding table 

#### 3. Count the number of nonmissing values 

In [4]:
title "Total Nonmissing Rows";
proc sql;
select count(*) as TotalRow format=comma16.,
       count(Airport_Code) as TotalAirportCode format=comma16.,
       count(Claim_Site) as TotalClaimSite format=comma16.,
       count(Disposition) as TotalDisposition format=comma16.,
       count(Claim_Type) as TotalClaimType format=comma16.,
       count(Date_Received) as TotalDateReceived format=comma16.,
       count(Incident_Date) as TotalIncidentDate format=comma16.
    from sq.claimsraw;
quit;
title;

TotalRow,TotalAirportCode,TotalClaimSite,TotalDisposition,TotalClaimType,TotalDateReceived,TotalIncidentDate
"42,528","42,179","42,295","33,469","42,303","42,528","42,528"


#### 4. View percentage of missing values in the columns

In [6]:
/*Create a macro variable with the total number of rows - 42,528*/

proc sql noprint;
select count(*)
    into :TotalRows trimmed
    from sq.claimsraw;
quit;
%put &=TotalRows;



title "Percentage of Missing Rows";
proc sql;
select 1-(count(Airport_Code)/&TotalRows) as PctAirportCode 
                                             format=percent7.2, 
       1-(count(Claim_Site)/&TotalRows) as PctClaimSite 
                                             format=percent7.2,
       1-(count(Disposition)/&TotalRows) as PctDisposition 
                                            format=percent7.2,
       1-(count(Claim_Type)/&TotalRows) as PctClaimType 
                                           format=percent7.2,
       1-(count(Date_Received)/&TotalRows) as PctDateReceived 
                                              format=percent7.2,
       1-(count(Incident_Date)/&TotalRows) as PctIncidentDate 
                                              format=percent7.2
    from sq.claimsraw;
quit;
title;

PctAirportCode,PctClaimSite,PctDisposition,PctClaimType,PctDateReceived,PctIncidentDate
0.82%,0.55%,21.3%,0.53%,0.00%,0.00%


#### 5. View the distinct values and frequencies

In [8]:
title "Column Distinct Values";

proc sql number;

/*Claim_Site*/
title2 "Column: Claim_Site";
select distinct Claim_Site
    from sq.claimsraw
    order by Claim_Site;
    
/*Disposition*/
title2 "Column: Disposition";
select distinct Disposition
    from sq.claimsraw
    order by Disposition;
    
/*Claim_Type*/
title2 "Column: Claim_Type"; 
select distinct Claim_Type
    from sq.claimsraw
    order by Claim_Type;
    
/*Date_Received*/
title2 "Column: Date_Received";
select distinct put(Date_Received, year4.) as Date_Received
    from sq.claimsraw
    order by Date_Received;
    
/*Incident_Date*/
title2 "Column: Incident_Date";
select distinct put(Incident_Date, year4.) as Incident_Date
    from sq.claimsraw
    order by Incident_Date;
quit;

title;

Row,Claim_Site
1,
2,Bus Station
3,Checked Baggage
4,Checkpoint
5,Motor Vehicle
6,Not Provided
7,Other
8,Pre-Check
Row,Disposition
1,


#### 6. Count the number of rows where Incident_Date occurs AFTER Date_Recieved - 65 rows*

In [10]:
title "Number of Claims where Incident Date Occurred After the Date Received";
proc sql;
select count(*) label="Date Needs Review"
    from sq.claimsraw
    where Incident_Date > Date_Received;
quit;
title;

Date Needs Review
65


#### 7.  Run a query to view all rows and columns where Incident_Date occurs AFTER Date_Received

In [12]:
proc sql;
select Claim_Number, Date_Received, Incident_Date 
    from sq.claimsraw
    where Incident_Date > Date_Received;
quit;

Claim_Number,Date_Received,Incident_Date
2018011146749,02JAN2017,10DEC2017
2018011646813,05JAN2017,12DEC2017
2018011046737,02JAN2017,06DEC2017
2018011146748,02JAN2017,08DEC2017
2018010346412,03JAN2017,01NOV2017
2018011646797,05JAN2017,19DEC2017
2018010346402,03JAN2017,16NOV2017
2018010946655,09JAN2017,31DEC2017
2018010946640,02JAN2017,23DEC2017
2018010946664,04JAN2017,19NOV2017


**Summary** Looks like all january is coded in the wrong year.

# Prepare Data

#### 8. Create a new table named Claims_NoDup that removes entirely duplicated rows. A duplicate claim exists if every value is duplicated.

In [14]:
proc sql;
create table Claims_NoDup as 
select distinct * 
    from sq.claimsraw;
quit;

33                                                         The SAS System                             15:27 Monday, January 25, 2021

342        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
342      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
343        


344        proc sql;
345        create table Claims_NoDup as
346        select distinct *
347            from sq.claimsraw;
NOTE: Table WORK.CLAIMS_NODUP created, with 42524 rows and 13 columns.

348        quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           0.11 seconds
      cpu time            0.15 seconds
      

349        
350        
351        ods html5 (id=saspy_internal) close;ods listing;
352        
34                                                         The SAS System                             15:27 Monday, January 25, 2021

353        


#### 9. Prepare data by performing the following tasks:
- a. Select the Claim_Number, Incident Date columns.
- b. Fix the 65 date issues you identified earlier by replacing the year 2017 with 2018 in the Date_Received column.
- c. Select the Airport_Name column
- d. Replace missing values in the Airport_Code column with the value Unknown.
- e1. Clean the Claim_Type column.
- e2. Clean the Claim_Site column.
- e3. Clean the Disposition column.
- f. Select the Close_Amount column.
- g. Select the State column and upper case all values.
- h. Select the StateName, County and City column. Proper case all values.
- i. Remove all rows where year of Incident_Date occurs after 2017. 
- j. Order the results by Airport_Code, Incident_Date.

In [15]:
proc sql;
create table work.Claims_Cleaned as
select 
/*a. Select the Claim_Number, Incident Date columns.*/
       Claim_Number label="Claim Number",
       Incident_Date format=date9. label="Incident Date",
/*b. Fix the 65 date issues you identified earlier by replacing the year 2017 with 2018 in the Date_Received column.*/
       case 
            when Incident_Date > Date_Received then intnx("year",Date_Received,1,"sameday")
            else Date_Received
       end as Date_Received label="Date Received" format=date9.,
/*c. Select the Airport_Name column*/
       Airport_Name label="Airport Name",
/*d. Replace missing values in the Airport_Code column with the value Unknown.*/
       case 
            when Airport_Code is null then "Unknown"
            else Airport_Code
       end as Airport_Code label="Airport Code",
/*e1. Clean the Claim_Type column.*/
       case 
           when Claim_Type is null then "Unknown"
           else scan(Claim_Type,1,"/","r") /*If I find a '/', scan and retrieve the first word*/
       end as Claim_Type label="Claim Type",
/*e2. Clean the Claim_Site column.*/
       case 
           when Claim_Site is null then "Unknown" 
           else Claim_Site 
       end as Claim_Site label="Claim Site",
/*e3. Clean the Disposition column.*/
       case 
           when Disposition is null then "Unknown"
           when Disposition="Closed: Canceled" then "Closed:Canceled"
           when Disposition="losed: Contractor Claim" then "Closed:Contractor Claim" 
           else Disposition
       end as Disposition,
/*f. Select the Close_Amount column.*/
       Close_Amount format=Dollar20.2 label="Close Amount", 
/*g. Select the State column and upper case all values.*/
       upcase(State) as State,
/*h. Select the StateName, County and City column. Proper case all values.*/
       propcase(StateName) as StateName label="State Name",
       propcase(County) as County,
       propcase(City) as City
    from Claims_NoDup
/*i. Remove all rows where year of Incident_Date occurs after 2017. */
    where year(Incident_Date) <= 2017
/*j. Order the results by Airport_Code, Incident_Date.*/
    order by Airport_Code, Incident_Date;
quit;

35                                                         The SAS System                             15:27 Monday, January 25, 2021

356        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
356      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
357        
358        proc sql;
359        create table work.Claims_Cleaned as
360        select
361        /*a. Select the Claim_Number, Incident Date columns.*/
362               Claim_Number label="Claim Number",
363               Incident_Date format=date9. label="Incident Date",
364        /*b. Fix the 65 date issues you identified earlier by replacing the year 2017 with 2018 in the Date_Received column.*/
365               case
366                    when Incident_Date > Date_Received then intnx("year",Date_Received,1,"sameday")
367                    else Date_Received
368               end as Date_Received label="Date 

**Validate the prepared data**

In [16]:
proc sql;
select count(*) as TotalRows
    from work.claims_cleaned;
quit;

title "SQL Distinct Values Validation";
proc sql;
/*Claim_Site*/
title2 "Column: Claim_Site";
select distinct Claim_Site
    from work.claims_cleaned
    order by Claim_Site;
/*Disposition*/
title2 "Column: Disposition";
select distinct Disposition
    from work.claims_cleaned
    order by Disposition;
/*Claim_Type*/
title2 "Column: Claim_Type"; 
select distinct Claim_Type
    from work.claims_cleaned
    order by Claim_Type;
/*Date_Received*/
title2 "Column: Date_Received";
select distinct put(Date_Received, year4.) as Date_Received
    from work.claims_cleaned
    order by Date_Received;
/*Incident_Date*/
title2 "Column: Incident_Date";
select distinct put(Incident_Date, year4.) as Incident_Date
    from work.claims_cleaned
    order by Incident_Date;
quit;
title;

TotalRows
42522
Claim Site
Bus Station
Checked Baggage
Checkpoint
Motor Vehicle
Not Provided
Other
Pre-Check
Unknown


#### 10. Use the work.Claims_Cleaned table to create a view named TotalClaims to count the number of claims for each Airport_Code and Year.

In [17]:
/*NOTE: View work.TOTALCLAIMS created, with 1491 rows and 5 columns.*/
proc sql;
create view TotalClaims as
select Airport_Code, Airport_Name, City, State, 
       year(Incident_date) as Year, 
       count(*) as TotalClaims
    from work.claims_cleaned
    group by Airport_Code, Airport_Name, City, State, calculated Year
    order by Airport_Code, Year;
quit;

39                                                         The SAS System                             15:27 Monday, January 25, 2021

457        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
457      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
458        
459        /*NOTE: View work.TOTALCLAIMS created, with 1491 rows and 5 columns.*/
460        proc sql;
461        create view TotalClaims as
462        select Airport_Code, Airport_Name, City, State,
463               year(Incident_date) as Year,
464               count(*) as TotalClaims
465            from work.claims_cleaned
466            group by Airport_Code, Airport_Name, City, State, calculated Year
467            order by Airport_Code, Year;
NOTE: SQL view WORK.TOTALCLAIMS has been defined.
468        quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           0.01 seconds
      cpu time  

In [19]:
proc sql outobs=10;
select *
    from TotalClaims;
quit;

Airport Code,Airport Name,City,State,Year,TotalClaims
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2013,9
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2014,3
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2015,4
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2016,5
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2017,3
ABI,Abilene Regional,Abilene,TX,2013,4
ABI,Abilene Regional,Abilene,TX,2014,6
ABI,Abilene Regional,Abilene,TX,2015,2
ABQ,Albuquerque International Sunport Airport,Albuquerque,NM,2013,39
ABQ,Albuquerque International Sunport Airport,Albuquerque,NM,2014,33


#### 11. Create a view name TotalEnplanements by using the OUTER UNION set operator to concatenate the enplanement2017 and boarding2013_2016 tables.

In [22]:
proc sql;
create view TotalEnplanements as
select LocID, Enplanement, input(Year,4.) as Year
    from sq.enplanement2017 
    outer union corr
select LocID, Boarding as Enplanement, Year
    from sq.boarding2013_2016
    order by Year, LocID;
quit;

49                                                         The SAS System                             15:27 Monday, January 25, 2021

533        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
533      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
534        


535        proc sql;
536        create view TotalEnplanements as
537        select LocID, Enplanement, input(Year,4.) as Year
538            from sq.enplanement2017
539            outer union corr
540        select LocID, Boarding as Enplanement, Year
541            from sq.boarding2013_2016
542            order by Year, LocID;
NOTE: SQL view WORK.TOTALENPLANEMENTS has been defined.
543        quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           0.01 seconds
      cpu time            0.00 seconds
      

544        
545        
546        ods html5 (id=saspy_internal) close;ods listing;
5

In [23]:
proc sql outobs=10;
select *
    from TotalEnplanements;
quit;

LocID,Enplanement,Year
0AK,"3,123",2013
16A,"3,652",2013
1G4,"140,886",2013
2A3,"2,336",2013
2A9,"3,622",2013
4A2,"2,500",2013
6R7,"2,870",2013
A61,"3,665",2013
ABE,"301,969",2013
ABI,"82,758",2013


#### 12. Create a table named work.ClaimsByAirport by joining the TotalClaims and TotalEnplanements views.

In [24]:
proc sql;
create table work.ClaimsByAirport as
select t.Airport_Code,t.Airport_Name, t.City, t.State, 
       t.Year, t.TotalClaims, e.Enplanement, 
       TotalClaims/Enplanement as PctClaims format=percent10.4
    from TotalClaims as t inner join
         TotalEnplanements as e
    on t.Airport_Code = e.LocID and 
       t.Year = e.Year
    order by Airport_Code, Year;
quit;

53                                                         The SAS System                             15:27 Monday, January 25, 2021

564        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
564      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
565        
566        proc sql;
567        create table work.ClaimsByAirport as
568        select t.Airport_Code,t.Airport_Name, t.City, t.State,
569               t.Year, t.TotalClaims, e.Enplanement,
570               TotalClaims/Enplanement as PctClaims format=percent10.4
571            from TotalClaims as t inner join
572                 TotalEnplanements as e
573            on t.Airport_Code = e.LocID and
574               t.Year = e.Year
575            order by Airport_Code, Year;
NOTE: Table WORK.CLAIMSBYAIRPORT created, with 1438 rows and 8 columns.

576        quit;
NOTE: PROCEDURE SQL used (Total process time):
     

In [25]:
proc sql outobs=10;
select *
    from ClaimsByAirport;
quit;

Airport Code,Airport Name,City,State,Year,TotalClaims,Enplanement,PctClaims
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2013,9,"301,969",0.0030%
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2014,3,"298,306",0.0010%
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2015,4,"320,544",0.0012%
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2016,5,"324,511",0.0015%
ABE,"Lehigh Valley International Airport, Allentown",Allentown,PA,2017,3,"328,914",0.0009%
ABI,Abilene Regional,Abilene,TX,2013,4,"82,758",0.0048%
ABI,Abilene Regional,Abilene,TX,2014,6,"93,656",0.0064%
ABI,Abilene Regional,Abilene,TX,2015,2,"86,000",0.0023%
ABQ,Albuquerque International Sunport Airport,Albuquerque,NM,2013,39,"2,477,783",0.0016%
ABQ,Albuquerque International Sunport Airport,Albuquerque,NM,2014,33,"2,354,184",0.0014%


# Create Final Report

In [32]:
*******************************************************************************************************;
*                                      ANALYZE AND EXPORT DATA                                        *;
* NOTE: Do not edit the code below. Run the program after all data preparation has been complete.     *;
* NOTE: The Claims_Cleaned and ClaimsByAirport tables must reside in the WORK library.                *;
* NOTE: The code below will create a report in the location of the PATH macro variable you set in the *;
*       StarterProgram.sas program. The report will be named FinalReport.html.                        *;
*******************************************************************************************************;


/******************************************BEGIN HTML REPORT************************************************/

/*Location of the prepared tables*/
%let inputLib=work;

/*Location of output files*/
%let path=C:\Users\pestyl\OneDrive - SAS\github repos\SAS Case Studies and Workshops\SQL Case Study - TSA Claims;

/*Close all default ODS output*/
ods _all_ close;

/*Output the HTML file to this location. Name the file 'FinalReport.html'*/
ods html5 file="FinalReport.html" path="&path";

/*Set up the HTML grid*/
ods layout gridded columns=3 rows=4 column_gutter=.25in row_gutter=.25in;

/*Set up the options*/
ods escapechar='^'; /*Used as an escape character for ODS TEXT*/
ods noproctitle; /*Remove all proctitles from the output*/
title;footnote; /*Clear any previously set titles or footnotes*/

/*Set colors and sizes for titles and text in the report*/
%let MainTitleColor=cx081d58;
%let MainTitleSize=28pt;
%let TitleColor=cx081d58;
%let TitleSize=16pt;
%let EmphasisNumbers=cx1d91c0;
%let EmphasisNumbersSize=26pt;
%let EmphasisText=14pt;


************************************************;
*Row 1                                         *;
************************************************;
ods region row=1 column=1 column_span=3;

/*Add the report's main title*/
ods text="^{style[textalign=c fontsize=&MainTitleSize color=&MainTitleColor]TSA Claims Case Study Check}";


************************************************;
*Row 2                                         *;
************************************************;
**************;
*COLUMN 1 & 2*;
**************;
ods region row=2 column=1 column_span=2;

/*Create a new table with the count of overall claims by year using the work.claims_cleaned table*/
proc sql;
create table TotalClaimsByYear as
select put(Incident_Date, year4.) as Year,
       count(*) as TotalClaims format=comma16.
    from &inputlib..Claims_Cleaned
    group by calculated Year
    order by Year;
quit;

/*Visualize the newly created table in a bar chart*/
ods graphics on /width=11in height=4.5in imagemap=on;
title h=&TitleSize color=&TitleColor "Total Claims by Year";
footnote "NOTE: Using the Claims Cleaned Table";
proc sgplot data=TotalClaimsByYear;
   vbar Year / response=TotalClaims colorresponse=TotalClaims
                dataskin=pressed
                barwidth=.8
                datalabel
                colormodel=(cxedf8b1 cx7fcdbb cx2c7fb8)
                tip=(Year TotalClaims);
   gradlegend / notitle;
   yaxis display=(noline noticks) grid label="Total Claims Filed";
quit;
title;
footnote;

**************;
*COLUMN 3    *;
**************;
ods region row=2 column=3;

/*Store the total enplanements value in a macro variable using the work.ClaimsByAirport table*/
proc sql noprint;
select sum(Enplanement) as TotalDateIssues format=comma16.
    into :TotalEnplanements trimmed
   from &inputlib..ClaimsByAirport;
quit;
ods text="^{style[just=c fontsize=&EmphasisText]Total Enplanements}";
ods text="^{style[just=r fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&TotalEnplanements}";

/*Store the total claims filed value in a macro variable*/
proc sql noprint;
select count(*) as TotalClaims format=comma16.
    into :TotalClaims trimmed
   from &inputlib..Claims_Cleaned;
quit;
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Total Claims Filed}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&TotalClaims}";

/*Store the percentage of claims filed value in a macro variable*/
     /*Step 1. Remove commas from the macro variables*/
%let TotalEnplanementsNum=%sysfunc(compress("&TotalEnplanements",","));
%let TotalClaimsNum=%sysfunc(compress("&TotalClaims",","));
     /*Step 2. Calculate the percentage of total claims by enplanements and store in a macro variable*/
data _null_;
   Total=&TotalClaimsNum/&TotalEnplanementsNum;
   PercentValue=put(Total,percent8.4);
   call symputx('PctClaimsTotal',PercentValue);
run;
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Percentage of Claims Filed by Enplanements}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&PctClaimsTotal}";

/*Store the average days of claim value in a macro variable*/
proc sql noprint;
select sum(Date_Received-Incident_Date)/count(*) as AvgDays format=5.1
    into :AvgDays
    from &inputlib..claims_cleaned;
quit;
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Average Time (in days) to File a Claim}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&AvgDays}";

/*Store average days of claim value in a macro variable*/
proc sql noprint;
select count(*) as TotalUnknownAirports format=comma5.
    into :TotalUnknownAirports
    from &inputlib..claims_cleaned
   where Airport_Code = "Unknown";
quit;
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Total Unknown Airports}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&TotalUnknownAirports}";


************************************************;
*Row 3                                         *;
************************************************;
**************;
*COLUMN 1    *;
**************;
ods region row=3 column=1;

/*Visualize the frequency of Claim_Type*/
ods graphics on /width=5.5in height=4in;
title h=&TitleSize color=&TitleColor "Types of Claims Filed (Claim_Type)";
proc sgplot data=&inputlib..claims_Cleaned;
   hbar Claim_Type / categoryorder=respdesc datalabel fillattrs=(color= cx7fcdbb);
   yaxis display=(nolabel noticks noline)
          valueattrs=(color=gray33 size=9pt);
    xaxis grid labelattrs=(color=gray33 size=9pt)
          valueattrs=(color=gray33 size=9pt);
run;
title;

**************;
*COLUMN 2    *;
**************;
ods region row=3 column=2;

/*Visualize the frequency of Disposition*/
ods graphics on /width=5.5in height=4in ;
title h=&TitleSize color=&TitleColor "Final Result of a Claim (Disposition)";
proc sgplot data=&inputlib..claims_Cleaned;
   hbar Disposition / categoryorder=respdesc
                      datalabel
                      fillattrs=(color=cxedf8b1);
   yaxis display=(nolabel noticks noline)
          valueattrs=(color=gray33 size=9pt);
    xaxis grid labelattrs=(color=gray33 size=9pt)
          valueattrs=(color=gray33 size=9pt);
run;
title;

**************;
*COLUMN 3    *;
**************;
ods region row=3 column=3;

/*Visualize the frequency of Claim_Site*/
ods graphics on /width=5.5in height=4in;
title h=&TitleSize color=&TitleColor "Location Site of a Claim (Claim_Site)";
proc sgplot data=&inputlib..Claims_Cleaned;
   hbar Claim_Site / categoryorder=respdesc datalabel fillattrs=(color=cx2c7fb8);
   yaxis display=(nolabel noticks noline)
          valueattrs=(color=gray33 size=9pt);
    xaxis grid labelattrs=(color=gray33 size=9pt)
          valueattrs=(color=gray33 size=9pt);
run;


************************************************;
*Row 4                                         *;
************************************************;
**************;
*COLUMN 1 & 2*;
**************;
ods region row=4 column=1 column_span=2;

/*Create a tablet view the top 20 airports by PctClaims with over 10,000,000 passengers*/
proc sql outobs=20;
create table top20airports as
select *
   from &inputlib..ClaimsByAirport
   where Enplanement > 10000000
   order by PctClaims desc;
quit;

/*Visualize the top20airports table*/
ods graphics on /width=12in height=4in;
title1 h=&TitleSize color=&TitleColor "Top 20 Airports by Highest Percetage of Claims Filed";
title2 h=&TitleSize color=&TitleColor "For Airports With More Than 10 Million Passengers";
proc sgplot data=top20airports;
   bubble x=Enplanement y=PctClaims size=TotalClaims / group=Airport_Code
                                                       datalabel=Airport_Code
                                                       transparency=.3
                                                       tip=(Airport_Name PctClaims Enplanement TotalClaims State)
                                                       datalabelattrs=(size=8 weight=bold);
   inset "Bubble size represents total number of claims" / position=topleft
                                                           textattrs=(size=8);
   yaxis grid label="Percentage of Claims Per Passengers";
   xaxis grid label="Total Passengers";
run;
title;

**************;
*COLUMN 3    *;
**************;
ods region row=4 column=3;

/*Store values of the airport with the highest PctClaims for airports over 10,000,000 passengers.*/
proc sql noprint;
select Airport_Name, Year, PctClaims format=percent8.4
    into :Name trimmed, :Year trimmed, :PctClaims trimmed
    from top20airports(obs=1);
quit;

ods text=";
ods text=";
/*Airport Name*/
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Airport with the Highest Percentage of Claims}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&Name}";
/*Year*/
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Year}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&Year}";
/*PctClaims*/
ods text="^{style[textalign=c fontsize=&EmphasisText color=&MainTitleColor]Percentage of Claims Per Passengers}";
ods text="^{style[textalign=c fontsize=&EmphasisNumbersSize color=&EmphasisNumbers]&PctClaims}";

/*End gridded layout*/
ods layout end;

/*Close output to HTML5*/
ods _all_ close;

/********************************************END REPORT************************************************/


69                                                         The SAS System                             15:27 Monday, January 25, 2021

2240       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
2240     ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
2241       
2242       *******************************************************************************************************;
2243       *                                      ANALYZE AND EXPORT DATA                                        *;
2244       * NOTE: Do not edit the code below. Run the program after all data preparation has been complete.     *;
2245       * NOTE: The Claims_Cleaned and ClaimsByAirport tables must reside in the WORK library.                *;
2246       * NOTE: The code below will create a report in the location of the PATH macro variable you set in the *;
2247       *       StarterProgram.sas progr